# Interactive Brain Visualization

This is an interactive tool for visualizing time series data from an ECoG experiment. You can do the following:
* Select electrodes with the menu list or mouse selection tool (use ctrl-L or ctrl-C and keep ctrl pressed, then use the mouse)
* Change the size and color of selected and unselected electrodes

In [1]:
import neuropythy as ny
import ipyvolume as ipv
from matplotlib import cm
import numpy as np
import scipy.io
import os
import wget

##unzip and load test subject data
from zipfile import ZipFile

#pip install --upgrade ipyvolume
# needs: pip install shapely
# pip install wget

## for dropdown menu
import ipywidgets as widgets
from ipywidgets import interact, interactive, interact_manual, GridspecLayout

# For loading data files
import h5py

from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [2]:
local_path = '../shablona/data'
local_path = '/Applications/freesurfer_v5/subjects/'
if not os.path.isdir(os.path.join(local_path,'test_subj')):
    # download example data with wget
    print('Downloading mesh and electrode data with wget')
    myfile = wget.download('https://zenodo.org/record/996814/files/Hamilton_et_al_img_pipe_test_subj.zip?download=1', local_path)
    # extract files
    with ZipFile(myfile, 'r') as zip_ref:
        #print(zip_ref.printdir())
        #zip_ref.extract('test_subj/surf/', local_path)
        #zip_ref.extract('test_subj/mri/', local_path)
        zip_ref.extractall(local_path)

# Get the functional data
data_dict = dict()
local_path = '../shablona/data'
with h5py.File('%s/test_subj_CV.hf5'%(local_path),'r') as f:
    stimulus_types = f['ecog'].keys()
    times = f['times'][:].ravel()
    #print(stimulus_types)
    for s in stimulus_types:
        data_dict[s] = f['ecog'][s][:]


In [3]:
#data_dict['CV_listen'].shape

In [14]:
# Get the electrode coordinates

local_path = '/Applications/freesurfer_v5/subjects/'
subj_path = local_path+'/test_subj'

if os.path.exists(subj_path):
    sub = ny.freesurfer_subject(subj_path)
    elecs = scipy.io.loadmat(os.path.join(subj_path,'elecs/TDT_elecs_all.mat'))
else:
    print('file not found?')
    sub = ny.freesurfer_subject('test_subj')
    elecs=scipy.io.loadmat('/Applications/freesurfer_v5/subjects/test_subj/elecs/TDT_elecs_all.mat')

#sub.hemis

In [15]:
def plot_activity(data_dict, times, condition, elec):
    '''
    Plot the activity for the selected electrode(s)
    Inputs:
    
        data_dict [dict] : 
        times [array-like] : 
        condition [str] : condition to plot (from data_dict.keys())
        elec [int] : selected electrode 
        
    '''
    if elec > data_dict[condition].shape[0]:
        data = np.zeros((data_dict[condition].shape[1],))
    else:
        data = data_dict[condition][elec,:].ravel()
    plt.plot(times, data)
    plt.axvline(color='k', linestyle='--', linewidth=1)
    plt.axhline(color='k', linestyle='--', linewidth=1)
    plt.xlabel('Time (s)')
    plt.ylabel('Z-scored HG')


In [16]:
# Probably shouldn't use this?  Seems like security would be an issue
upload_widget = widgets.FileUpload(
    accept='',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # True to accept multiple files upload else False
)

In [20]:
anatomical_areas = [area[0] for area in np.unique(elecs['anatomy'][:,3])]
#                      selected_anatomy = anatomical_areas,

elec_select = widgets.SelectMultiple(
    options= elecs['eleclabels'][:,0],
    value=['G1'],
    #rows=10,
    description='Electrode(s)',
    disabled=False
)

#@interact

def select_parameters(selected_elec = elecs['eleclabels'][:,0], 
                      selected_type = ['raw','not raw'],
                      selected_stimulus = [k for k in data_dict.keys()],
                      selected_time = (0, 10, 10000)):
    elecs_scatter.selected = np.where(elecs['eleclabels'][:,0] == selected_elec)
    fig = plot_activity(data_dict, times, condition = selected_stimulus, elec = elecs_scatter.selected)
    return
    #return fig

#elecs_scatter.selected

w = interactive(select_parameters)

In [21]:
# Plot the brain
#ipf = ny.cortex_plot(sub.rh.pial_surface, color=sub.rh.properties['BA44_weight'], cmap='Reds', alpha=0.7)

# Make the brain white but slightly transparent (alpha=0.5)
clr = np.ones((sub.rh.pial_surface.vertex_count,4))
clr[:,3] = 0.5

# Initialize ipyvolume figure
ipv.figure()
ipf = ny.cortex_plot(sub.rh.pial_surface, color=clr, alpha=0.5, underlay=None, mesh_alpha = 0.5)

# Get the lasso selector
ipv.selector_default()

# Plot the electrodes as spheres
elecs_scatter = ipv.scatter(elecs['elecmatrix'][:,0],elecs['elecmatrix'][:,1],elecs['elecmatrix'][:,2], 
                            marker='sphere', selected=[0,1,2,3,4], color_selected='cornflowerblue', 
                            color='coral', size_selected = 3.0)

from ipywidgets import FloatSlider, ColorPicker, VBox, jslink

style = {'description_width': '100px'}

size = FloatSlider(min=0, max=10, step=0.1, description='Unselected size', style=style)
size_selected = FloatSlider(min=0, max=10, step=0.1, description='Selected size', style=style)
color = ColorPicker(concise=True, description='Unselected', style=style)
color_selected = ColorPicker(concise=True, description='Selected', style=style)
jslink((elecs_scatter, 'size'), (size, 'value'))
jslink((elecs_scatter, 'size_selected'), (size_selected, 'value'))
jslink((elecs_scatter, 'color'), (color, 'value'))
jslink((elecs_scatter, 'color_selected'), (color_selected, 'value'))
#VBox([ipv.gcc(), size, size_selected, color, color_selected])

#ipv.show()

grid = GridspecLayout(15, 2)
grid[:, 0] = ipf
grid[0, 1] = size
grid[1, 1] = size_selected
grid[2, 1] = color
grid[3, 1] = color_selected
grid[4:, 1] = w
#grid[4, 1] = fig

grid

Output()

GridspecLayout(children=(Figure(camera=PerspectiveCamera(fov=0.644570721372708, position=(0.0, -100.0, 0.0), q…